In [4]:
import pandas as pd
import regex as re

# Load scraped messages
df = pd.read_csv("data/scrapped/telegram_data.csv")
print(df.columns)

def clean_amharic_text(text):
    if pd.isnull(text):
        return ""

    # Remove emojis & symbols
    text = re.sub(r'[\p{So}\p{Sk}\p{Mn}]', '', text)

    # Remove Latin letters, digits, and URLs
    text = re.sub(r'(https?:\/\/\S+|www\.\S+)', '', text)
    text = re.sub(r'[A-Za-z0-9]', '', text)

    # Normalize Amharic punctuation
    text = text.replace('፡', ' ').replace('።', '.').replace('፣', ',')
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply cleaning
df['clean_text'] = df['Message'].apply(clean_amharic_text)

# Save cleaned output
df.to_csv("cleaned_telegram_data.csv", index=False)


Index(['Channel Title', 'Channel Username', 'ID', 'Message', 'Date',
       'Media Path'],
      dtype='object')


Tokenization

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification


tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained(tokenizer)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

df = pd.read_csv("cleaned_telegram_data.csv")
token = tokenizer.tokenize(df['clean_text'])
entities = ner_pipeline(df['clean_text'])

for entity in entities:
    print(f"Entity: {entity['word']}, Score: {entity['score']:.2f}, Label: {entity['entity_group']}")



ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
